In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
base_dir = 'data/KDEF_and_AKDEF/KDEF/'

In [3]:
import os
# 显示几个随机增强后的训练图像
from keras.preprocessing import image 

In [4]:
# 将文件名读入list中
def read(filename):
    data = []
    file = open(filename, 'r')
    for line in file:
        data.append(line.replace('\n',''))
    file.close()
    return data

In [5]:
training_data = read('data/train.txt')
test_data = read('data/test.txt')
validation_data = read('data/validation.txt')

In [6]:
len(training_data), len(test_data), len(validation_data)

(2900, 1000, 1000)

In [7]:
import numpy as np
# 从文本中提取数据出来
def process(data):
    path = [item.split(',')[0] for item in data]
    expression = [int(item.split(',')[1]) for item in data]
    angle = [int(item.split(',')[2]) for item in data]
    return path, expression, angle

# one-hot 编码
def to_one_hot(labels, dimension=1):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [8]:
training_path, training_expression, training_angle = process(training_data)
test_path, test_expression, test_angle = process(test_data)
validation_path, validation_expression, validation_angle = process(validation_data)

In [9]:
# 可视化data augmentation图片
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')


img = image.load_img(base_dir + training_path[0], target_size=(200, 200)) # 读取图像并调整大小

x = image.img_to_array(img) # 将其转化为(img.size[0], img.size[1], 3)的Numpy数组

x = x.reshape((1,) + x.shape) # 将其转化为(1, img.size[0], img.size[1], 3)的Numpy

import matplotlib.pyplot as plt

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot=plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()
    

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [10]:
# 将数据分放
data_dir = 'data/'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

label_range = np.unique(training_expression)

# training data dictionary
train_dir = os.path.join(data_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    for i in label_range:
        train_i_dir = os.path.join(train_dir, str(i))
        if not os.path.exists(train_i_dir):
            os.mkdir(train_i_dir)
            
# test data dictionary
test_dir = os.path.join(data_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
    for i in label_range:
        test_i_dir = os.path.join(test_dir, str(i))
        if not os.path.exists(test_i_dir):
            os.mkdir(test_i_dir)
            
# validation data dictionary
validation_dir = os.path.join(data_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    for i in label_range:
        validation_i_dir = os.path.join(validation_dir, str(i))
        if not os.path.exists(validation_i_dir):
            os.mkdir(validation_i_dir)

In [11]:
# copy file to the dictionary
import shutil

# training data copy
for path, expression in zip(training_path, training_expression):
    file = path.split('/')[1]
    src = os.path.join(base_dir, path)
    dst = os.path.join(train_dir, str(expression), file)
    shutil.copyfile(src, dst)

# test data copy
for path, expression in zip(test_path, test_expression):
    file = path.split('/')[1]
    src = os.path.join(base_dir, path)
    dst = os.path.join(test_dir, str(expression), file)
    shutil.copyfile(src, dst)
    
for path, expression in zip(validation_path, validation_expression):
    file = path.split('/')[1]
    src = os.path.join(base_dir, path)
    dst = os.path.join(validation_dir, str(expression), file)
    shutil.copyfile(src, dst)

In [12]:
# 引入预训练模型
from keras.applications import  ResNet50
conv_base = ResNet50(include_top=False, 
                     weights='imagenet',
                     input_shape=(224, 224, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [13]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [14]:
train_dir

'data/train'

In [15]:
from tqdm import tqdm
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 2048))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in tqdm(generator):
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            #注意，这些生成器在循环中不断生成数据，所以你必须在读取完所有图像后终止循环
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2900)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

  0%|          | 0/145 [00:00<?, ?it/s]

Found 2900 images belonging to 7 classes.



  0%|          | 0/50 [00:00<?, ?it/s]

Found 1000 images belonging to 7 classes.



 98%|█████████▊| 49/50 [00:54<00:01,  1.11s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Found 1000 images belonging to 7 classes.




  2%|▏         | 1/50 [00:01<00:55,  1.13s/it]

  4%|▍         | 2/50 [00:02<00:53,  1.12s/it]

  6%|▌         | 3/50 [00:03<00:52,  1.12s/it]

  8%|▊         | 4/50 [00:04<00:51,  1.12s/it]

 10%|█         | 5/50 [00:05<00:50,  1.12s/it]

 12%|█▏        | 6/50 [00:06<00:49,  1.12s/it]

 14%|█▍        | 7/50 [00:07<00:48,  1.12s/it]

 16%|█▌        | 8/50 [00:08<00:47,  1.12s/it]

 18%|█▊        | 9/50 [00:10<00:45,  1.12s/it]

 20%|██        | 10/50 [00:11<00:44,  1.12s/it]

 22%|██▏       | 11/50 [00:12<00:43,  1.12s/it]

 24%|██▍       | 12/50 [00:13<00:42,  1.12s/it]

 26%|██▌       | 13/50 [00:14<00:41,  1.12s/it]

 28%|██▊       | 14/50 [00:15<00:40,  1.11s/it]

 30%|███       | 15/50 [00:16<00:39,  1.12s/it]

 32%|███▏      | 16/50 [00:17<00:38,  1.12s/it]

 34%|███▍      | 17/50 [00:19<00:36,  1.12s/it]

 36%|███▌      | 18/50 [00:20<00:35,  1.12s/it]

 38%|███▊      | 19/50 [00:21<00:34,  1.12s/it]

 40%|████      | 20/50 [00:22<00:33,  1.12s/it]

 42%|████▏     | 21/50 [00:

In [16]:
train_features.shape, train_labels

((2900, 7, 7, 2048), array([6., 3., 4., ..., 5., 2., 6.]))

In [17]:
train_features = np.reshape(train_features, (2900, 7 * 7 * 2048))
validation_features = np.reshape(validation_features, (1000, 7 * 7 * 2048))
test_features = np.reshape(test_features, (1000, 7 * 7 * 2048))

In [18]:
# one_hot 编码
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)
one_hot_validation_labels = to_categorical(validation_labels)

In [19]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, one_hot_train_labels,
                    epochs=10,
                    batch_size=20,
                    validation_data=(validation_features, one_hot_validation_labels))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2900 samples, validate on 1000 samples
Epoch 1/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4543 - acc: 0.8455 - val_loss: 0.4121 - val_acc: 0.8571
Epoch 2/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4448 - acc: 0.8521 - val_loss: 0.4131 - val_acc: 0.8571
Epoch 3/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4419 - acc: 0.8512 - val_loss: 0.4131 - val_acc: 0.8571
Epoch 4/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4460 - acc: 0.8529 - val_loss: 0.4158 - val_acc: 0.8571
Epoch 5/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4400 - acc: 0.8515 - val_loss: 0.4160 - val_acc: 0.8571
Epoch 6/10
2900/2900 [==============================] - 16s 5ms/step - loss: 0.4404 - acc: 0.8523 - val_loss: 0.4142 - val_acc: 0.8571
Epoch 7/10
2900/2900 [==========================

In [20]:
result = model.evaluate(test_features, one_hot_test_labels)

1000/1000 [==============================] - 1s 621us/step


In [21]:
result

[0.41484973502159117, 0.8571429252624512]